In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
!pip install -qU langchain_openai>=0.3.4 \
langchain_huggingface \
langchain_core>=0.3.34 \
langchain>=0.3.18 \
langchain_community>=0.3.17 \
langchain-text-splitters>=0.3.6 \
datasets>=3.2.0 \
faiss-cpu \
python-pptx==1.0.2 \
nltk==3.9.1 \
pymupdf \
beautifulsoup4 \
lxml \
sentence_transformers \
pyarrow \
langchain_qdrant

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [4]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [5]:
  import json

  with open('/content/drive/MyDrive/MedicarePilot/json_files/test_dataset.jsonl', 'r') as f:
    test_dataset = json.load(f)

  with open('/content/drive/MyDrive/MedicarePilot/json_files/training_dataset.jsonl', 'r') as f:
    train_dataset = json.load(f)

  with open('/content/drive/MyDrive/MedicarePilot/json_files/val_dataset.jsonl', 'r') as f:
    val_dataset = json.load(f)

In [6]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [7]:
BATCH_SIZE = 10

In [8]:

corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [9]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [10]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [11]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [12]:
EPOCHS = 10

In [13]:
import wandb
wandb.init(mode="disabled")

In [14]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.695000,0.885000,0.940000,0.990000,0.695000,0.295000,0.188000,0.099000,0.695000,0.885000,0.940000,0.990000,0.848328,0.802220,0.803091
91,No log,No log,0.725000,0.925000,0.955000,1.000000,0.725000,0.308333,0.191000,0.100000,0.725000,0.925000,0.955000,1.000000,0.870466,0.828038,0.828038
100,No log,No log,0.695000,0.915000,0.955000,1.000000,0.695000,0.305000,0.191000,0.100000,0.695000,0.915000,0.955000,1.000000,0.858480,0.811976,0.811976
150,No log,No log,0.720000,0.930000,0.945000,1.000000,0.720000,0.310000,0.189000,0.100000,0.720000,0.930000,0.945000,1.000000,0.870083,0.827429,0.827429
182,No log,No log,0.715000,0.930000,0.965000,1.000000,0.715000,0.310000,0.193000,0.100000,0.715000,0.930000,0.965000,1.000000,0.869029,0.825774,0.825774
200,No log,No log,0.690000,0.935000,0.960000,1.000000,0.690000,0.311667,0.192000,0.100000,0.690000,0.935000,0.960000,1.000000,0.858672,0.811879,0.811879
250,No log,No log,0.705000,0.925000,0.955000,1.000000,0.705000,0.308333,0.191000,0.100000,0.705000,0.925000,0.955000,1.000000,0.861261,0.815859,0.815859
273,No log,No log,0.700000,0.915000,0.975000,1.000000,0.700000,0.305000,0.195000,0.100000,0.700000,0.915000,0.975000,1.000000,0.859028,0.812720,0.812720
300,No log,No log,0.725000,0.910000,0.955000,1.000000,0.725000,0.303333,0.191000,0.100000,0.725000,0.910000,0.955000,1.000000,0.870627,0.828506,0.828506
350,No log,No log,0.725000,0.925000,0.975000,0.995000,0.725000,0.308333,0.195000,0.099500,0.725000,0.925000,0.975000,0.995000,0.870386,0.829103,0.829558


In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
hf_username = "vivnatan"

In [17]:

import uuid

model.push_to_hub(f"{hf_username}/snowflake-arctic-embed-l-medicare")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/vivnatan/snowflake-arctic-embed-l-medicare/commit/603b29d0a64b2399dbe2884b3c28dfad0376f656'

## Evaluating embeddings - Vibe check

In [23]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

import tqdm

In [24]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

In [25]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 200/200 [01:11<00:00,  2.81it/s]


In [26]:
te3_results_df = pd.DataFrame(te3_results)

In [27]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

np.float64(0.955)

In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 200/200 [00:04<00:00, 45.08it/s]


In [29]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [30]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

np.float64(0.73)

In [31]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="vivnatan/snowflake-arctic-embed-l-medicare")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/43.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

100%|██████████| 200/200 [00:04<00:00, 45.12it/s]


In [32]:
finetune_results_df = pd.DataFrame(finetune_results)

In [33]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

np.float64(0.935)

In [4]:
!pip install -qU langchain-community==0.3.14 \
                 langchain-openai==0.2.14 \
                 unstructured==0.16.12 \
                 langgraph==0.2.61 \
                 langchain-qdrant==0.2.0 \
                 ragas==0.2.10 \
                 pymupdf \
                 cohere \
                 langchain_cohere \
                 langchain_huggingface \
                 langchain_core>=0.3.34 \
                 langchain>=0.3.18 \
                 langchain-text-splitters>=0.3.6 \
                 datasets>=3.2.0 \
                 sentence_transformers \
                 pyarrow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.


In [5]:
import os
from pathlib import Path
from tqdm.notebook import tqdm

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema import Document

PDF_ROOT = Path("/content/drive/MyDrive/MedicarePilot/pdf_files")

pdf_paths = list(PDF_ROOT.rglob("*.pdf"))

# Load PDFs into Documents
pdf_docs = []
errors = []

for pdf_path in tqdm(pdf_paths, desc="Loading PDFs"):
    try:
        loader = PyMuPDFLoader(str(pdf_path))
        # load() returns a list of Document, one per page by default
        page_docs = loader.load()
        # enrich metadata with filename
        for doc in page_docs:
            # ensure metadata has source and page info
            meta = dict(doc.metadata)
            meta["source"] = str(pdf_path)
            # if loader provides 'page' or 'page_number', keep it; otherwise derive
            if "page" not in meta:
                # infer page index from doc.content order
                # PyMuPDFLoader sets metadata["page"] for each Document
                pass
            doc.metadata = meta  # type: ignore
        pdf_docs.extend(page_docs)
        print(f"  ✅ Loaded {len(page_docs)} pages from {pdf_path.name}")
    except Exception as e:
        errors.append((pdf_path.name, str(e)))
        print(f"  ⚠️  Failed to load {pdf_path.name}: {e}")

Loading PDFs:   0%|          | 0/1 [00:00<?, ?it/s]

  ✅ Loaded 128 pages from 10050-medicare-and-you.pdf


/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 126 of document /content/drive/MyDrive/MedicarePilot/pdf_files/10050-medicare-and-you.pdf
  warnings.warn(


In [6]:
import tiktoken

from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

In [7]:
text_splitter2 = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = tiktoken_len,
)

In [8]:
split_chunks2 = text_splitter2.split_documents(pdf_docs)

In [12]:
!pip uninstall numpy
!pip install -qU numpy==1.24.3
!pip uninstall -y transformers sentence_transformers scikit-learn scipy
!pip install -qU transformers sentence_transformers scikit-learn scipy

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 121.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
albumen

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings4 = HuggingFaceEmbeddings(model_name="vivnatan/snowflake-arctic-embed-l-medicare")

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a seasoned health insurance advisor with over 30 years in the industry.
You have tremendous knowledge about the industry and the different players.
You keep track of the changes in regulations and you are here to advise the user with their questions or concerns.
If there are any industry jargons in your responses make them easy to understand and provide details of what they are.
Wherever available provide references.
If you are unable to get the answer from the context provided, just say that you don't know. Don't try to make up an answer.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [30]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [31]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [32]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client4 = QdrantClient(":memory:")

client4.create_collection(
    collection_name="medicare_signup4",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)



vector_store4 = QdrantVectorStore(
    client=client4,
    collection_name="medicare_signup4",
    embedding=embeddings4,
)

In [33]:
_ = vector_store4.add_documents(documents=split_chunks2)

In [34]:
retriever4 = vector_store4.as_retriever(search_kwargs={"k": 5})

In [35]:
def retrieve4(state):
  retrieved_docs = retriever4.invoke(state["question"])
  return {"context" : retrieved_docs}

In [36]:
graph_builder4 = StateGraph(State).add_sequence([retrieve4, generate])
graph_builder4.add_edge(START, "retrieve4")
graph4 = graph_builder4.compile()

In [37]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [38]:
import pandas as pd
from ragas import EvaluationDataset
import json
import ast

dataset = pd.read_csv("/content/drive/MyDrive/SDG.csv")

# Assuming 'reference_contexts' is a string representation of a list
# Convert it to an actual list using ast.literal_eval or similar method
# json.loads expects a valid JSON string, which might not be the case here.
# ast.literal_eval is safer for evaluating strings containing Python literals.
dataset['reference_contexts'] = dataset['reference_contexts'].apply(lambda x: ast.literal_eval(x))

# Create the EvaluationDataset
evaluation_dataset = EvaluationDataset.from_pandas(dataset)

In [43]:
for index, test_row in dataset.iterrows():
  response4 = graph4.invoke({"question" : test_row['user_input']}) # Access 'user_input' directly
  test_row['response'] = response4["response"] # Create new columns for response and retrieved_contexts
  test_row['retrieved_contexts'] = [context.page_content for context in response4["context"]]

In [46]:
response4

{'question': 'How do I get new Medicare card if lost or damaged?',
 'context': [Document(metadata={'source': '/content/drive/MyDrive/MedicarePilot/pdf_files/10050-medicare-and-you.pdf', 'file_path': '/content/drive/MyDrive/MedicarePilot/pdf_files/10050-medicare-and-you.pdf', 'page': 107, 'total_pages': 128, 'format': 'PDF 1.6', 'title': 'Medicare and You Handbook 2025', 'author': 'Centers for Medicare and Medicaid Services', 'subject': 'Medicare and You Handbook 2025', 'keywords': 'Medicare and You Handbook 2025; National version', 'creator': 'Adobe InDesign 20.1 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20250122134726-05'00'", 'modDate': "D:20250214142943-05'00'", 'trapped': '', '_id': '83170b8816844923a2c4d2e04c43119c', '_collection_name': 'medicare_signup4'}, page_content='108 Section 9: Find helpful contacts and more information\n108\nIf you need a new copy of your Medicare card\nIf you need to replace your card because it’s \ndamaged or lost, visit Med

In [48]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset)

In [50]:
evaluation_dataset

EvaluationDataset(features=['user_input', 'reference_contexts', 'reference'], len=8)

In [49]:
#Chunk size - 1000 tokens
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result4 = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result4

ValueError: The metric [context_recall] that is used requires the following additional columns ['retrieved_contexts'] to be present in the dataset.